In [ ]:
import zipfile

In [ ]:
fantasy_zip = zipfile.ZipFile('./Stock_Dataset(2017_07_06).zip')
fantasy_zip.extractall('./data')

In [16]:
import pandas as pd
from pandas import Series, DataFrame
from sklearn import preprocessing
import numpy as np
import tensorflow as tf


In [2]:
!pip install xlrd

mxnet-cu80 1.1.0 has requirement numpy<=1.13.3, but you'll have numpy 1.14.3 which is incompatible.


In [33]:
!pip install openpyxl

    100% |████████████████████████████████| 174kB 2.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for openpyxl ... done
  Stored in directory: /home/U2017042/.cache/pip/wheels/d3/75/83/415a5bbed8366f66a87f7bd0fc28f7df6e13982960f3e5a7ab
  Running setup.py bdist_wheel for et-xmlfile ... done
  Stored in directory: /home/U2017042/.cache/pip/wheels/2a/77/35/0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile
mxnet-cu80 1.1.0 has requirement numpy<=1.13.3, but you'll have numpy 1.14.3 which is incompatible.


In [3]:
corporations = pd.read_excel('./corporations.xlsx')

In [ ]:
corporations 

In [4]:
stock_corps = corporations.query("상장일<'2005-01-01'  ")[['회사명', '종목코드']]
len(stock_corps)

1068

In [35]:
# 상세 데이터를 가져온다.
def get_stock_datail(row) :
    code = row['종목코드']
    code = format(code, "06d");
    return pd.read_csv('./data/' + code + '.csv')

# matrix 데이터로 변경한다.
def to_ndarray(cols_data) :
    if isinstance(cols_data, Series):
        return np.reshape(list(cols_data), (-1,1))
    elif isinstance(cols_data, DataFrame):
        return cols_data.as_matrix()

# 컬럼을 스케일링을 시킨다.
def get_scaled_cols(data, column_name) :
    scale_data = to_ndarray(data[column_name])
    scaler = preprocessing.MinMaxScaler()
    return scaler.fit_transform(scale_data);

# 데이터를 스케일링 시킨다.
def get_scaled_data(data) :
    scaled_data = data
    scaled_data['Close'] = get_scaled_cols(scaled_data, 'Close')
    scaled_data['Open'] = get_scaled_cols(scaled_data, 'Open')
    scaled_data['High'] = get_scaled_cols(scaled_data, 'High')
    scaled_data['Low'] = get_scaled_cols(scaled_data, 'Low')
    scaled_data['Volume'] = get_scaled_cols(scaled_data, 'Volume')
    return scaled_data;

# RNN을 위한 데이터로 만든다. 
def get_dataXY(data, train_params) :
    x = to_ndarray(data[['Open', 'High', 'Low', 'Volume', 'Close']])
    y = to_ndarray(data['Close'])
    
    dataX = []
    dataY = []
    seq_length = train_params['seq_length']
    for i in range(0, len(y) - seq_length):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length]  # Next close price
        #print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return dataX, dataY

# train 및 test 데이터로 나눈다.
def split_train_test(dataX, dataY) :
    train_size = int(len(dataY) * 0.7)
    test_size = len(dataY) - train_size
    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:len(dataX)])
    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:len(dataY)])
    return trainX, trainY, testX, testY

# train, test데이터로 만든다.
def get_train_test(data, train_params) :
    scaled_data = get_scaled_data(data)
    dataX, dataY = get_dataXY(scaled_data, train_params)
    return split_train_test(dataX, dataY)

# 텐스플로우 변수관계 그래프롤 그린다.
def draw_graph(train_params) :
    
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, [None, train_params['seq_length'], 
                                    train_params['data_dim']])
    Y = tf.placeholder(tf.float32, [None, 1])
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=train_params['hidden_dim'], 
                                        state_is_tuple=True, 
                                        activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32 )
    Y_pred = tf.contrib.layers.fully_connected(
        outputs[:, -1], train_params['output_dim'], activation_fn=None)  # We use the last cell's output

    # cost/loss
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    # optimizer
    optimizer = tf.train.AdamOptimizer(train_params['learning_rate'])
    train = optimizer.minimize(loss)

    # RMSE
    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])
    rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))
    return {
        'X': X,
        'Y': Y,
        'train': train,
        'loss' : loss,
        'trainX': trainX,
        'trainY': trainY,
        'testX': testX,
        'testY': testY,
        'Y_pred': Y_pred,
        'targets': targets,
        'rmse' : rmse,
        'predictions': predictions
    }

# 학습을 시킨다.
def let_training(train_params, graph_params) :
    X = graph_params['X']
    Y = graph_params['Y']
    train = graph_params['train']
    loss = graph_params['loss']
    trainX = graph_params['trainX']
    trainY = graph_params['trainY']
    testX = graph_params['testX']
    testY = graph_params['testY']
    Y_pred = graph_params['Y_pred']
    targets = graph_params['targets']
    rmse = graph_params['rmse']
    predictions = graph_params['predictions']
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(train_params['iterations']):
            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
        
        test_predict = sess.run(Y_pred, feed_dict={X: testX})
        rmse_val = sess.run(rmse, feed_dict={targets: testY, predictions: test_predict})
        #print("RMSE: {}".format(rmse_val))
        return rmse_val

# 그래프를 그리고 학습을 시킨다.    
def let_leaning(train_params):
    graph_params = draw_graph(train_params)
    rmse_val = let_training(train_params, graph_params)
    return rmse_val

def save_excel(data, columns, file_name):
    df_data = pd.DataFrame(data, columns=columns)
    writer = pd.ExcelWriter(file_name)
    df_data.to_excel(writer,'Sheet1', index=False)
    writer.save()
    

In [6]:
# train Parameters
train_params = {
    'seq_length' : 10,
    'data_dim' : 5,
    'hidden_dim' : 10,
    'output_dim' : 1,
    'learning_rate' : 0.001,
    'iterations' : 1000,
};

In [ ]:
comp_rmses = []
for idx, row in stock_corps.iterrows():
    data = get_stock_datail(row)
    trainX, trainY, testX, testY = get_train_test(data, train_params)
    rmse_val = let_leaning(train_params)
    if idx == 0 :
        print('종목코드', '회사명', 'RMSE')
    print(row['종목코드'], row['회사명'], rmse_val)
    comp_rmses.append([row['종목코드'], row['회사명'], rmse_val])
 
save_excel(comp_rmses, ['code', 'name', 'RMSE'], 'comp_rmses.xlsx')


/opt/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/venv/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


종목코드 회사명 RMSE
1460 BYC 0.044718813
79160 CJ CGV 0.05417542
5830 DB손해보험 0.033352792
69730 DSR제강 0.026194276
9440 KC그린홀딩스 0.00856093
1940 KISCO홀딩스 0.019114956
23150 MH에탄올 0.0067128995
34310 NICE 0.03614047
36530 S&T홀딩스 0.011087676
17670 SK텔레콤 0.03586618
12200 계양전기 0.032947484
1290 골든브릿지증권 0.008371822
24110 기업은행 0.020934915
4540 깨끗한나라 0.034879055
1260 남광토건 0.015140646
25860 남해화학 0.0088789165
58730 다스코 0.06481042
6370 대구백화점 0.027928116
3540 대신증권 0.011697889
1130 대한제분 0.02608476
3490 대한항공 0.014037913
5880 대한해운 0.004999791
69460 대호에이엘 0.024304973
24900 덕양산업 0.087300636
24090 디씨엠 0.032771893
4990 롯데지주 0.060686268
11170 롯데케미칼 0.02081672
27740 마니커 0.0040602223
8560 메리츠종금증권 0.010705858
9680 모토닉 0.00565537
2760 보락 0.04696684
3850 보령제약 0.02364444
79660 사조해표 0.031118477
2170 삼양통상 0.08896621
11230 삼화전자공업 0.012545224
41650 상신브레이크 0.11968635
58650 세아홀딩스 0.024687674
33530 세종공업 0.018105648
4430 송원산업 0.023688648
2870 신풍제지 0.053489555
1770 신화실업 0.02708645
4080 신흥 0.011987974
3560 아이에이치큐 0.014436547
12750 

60260 뉴보텍 0.009318045
17650 대림제지 0.05823028
78600 대주전자재료 0.012218116
65690 대진디엠피 0.015635202
30350 드래곤플라이 0.021807963
43360 디지아이 0.014990394
47440 레이젠 0.040060047
60240 룽투코리아 0.06566647
58470 리노공업 0.035476852
33200 모아텍 0.040348962
29480 바른테크놀로지 0.015562585
73640 삼원테크 0.013189993
32280 삼일 0.03815718
42600 새로닉스 0.017977508
12860 서연전자 0.024169968
24830 세원물산 0.033678092
53110 소리바다 0.026441965
37760 쎄니트 0.006051793
66790 씨씨에스 0.010044795
36170 씨티엘 0.014916848
65420 에스아이리소스 0.013140073
39440 에스티아이 0.043388847
3800 에이스침대 0.030343508
50320 에이앤티앤 0.049195558
44780 에이치케이 0.021481188
38870 에코바이오 0.03168857
48830 엔피케이 0.031027855
33160 엠케이전자 0.016942997
37400 우리조명 0.025606986
46940 우원개발 0.07984914
65680 우주일렉트로 0.013208151
44340 위닉스 0.038517702
38620 위즈코프 0.010526283
11320 유니크 0.035793655
35810 이지바이오 0.059021745
51370 인터플렉스 0.0141104385
33050 제이엠아이 0.011887641
51160 지어소프트 0.030762691
18290 지엠피 0.040433556
50110 캠시스 0.011464979
25880 케이씨피드 0.0054010022
69110 코스온 0.041290984
51360 토비스 0.029803548
389